In [0]:
import numpy as np
import pickle
import cv2
from os import listdir
from sklearn.preprocessing import LabelBinarizer
import tensorflow as tf
from tensorflow.python import keras

In [0]:
# Load the Drive helper and mount
from google.colab import drive

# This will prompt for authorization.
drive.mount('/content/drive')
#4/EAHvxOwTUUc5P4f054p_XPb6LUtIp67TjDF6lZaN8qOKZAuMbtL9Fow

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
!unzip "/content/drive/My Drive/BEPROJ/dataset/c3d_train/c3d.zip"
!ls Anomaly -l | egrep -c '^-'

In [0]:
import csv
import configparser
import collections
import time
import csv
import os
from os import listdir
import skimage.transform
from skimage import color
from os.path import isfile, join
import numpy as np
import numpy
from datetime import datetime
from os.path import basename
import glob
import matplotlib.pyplot as plt
from IPython.display import clear_output
import math

In [0]:
# count = 0
# image_list, label_list = [], []

# Load Training Dataset and label training videos
import re 

def get_name(name):
  pattern = '[0-9]'
  return re.sub(pattern, '', name)

def load_dataset_Train_batch(AbnormalPath):

    batchsize=30            # Each batch contain 30 videos.
    n_exp=int(batchsize/2)  # 15 normal and 15 road accident videos

    Num_abnormal = 144  # number of road accident videos in Training set.


    # We assume the features of abnormal videos and normal videos are located in two different folders.
    Abnor_list_iter = np.random.permutation(Num_abnormal)
    # Abnor_list_iter = Abnor_list_iter[Num_abnormal-n_exp:] # Indexes for randomly selected Abnormal Videos

    AllVideos_Path = AbnormalPath
    def listdir_nohidden(AllVideos_Path):  # To ignore hidden files
        file_dir_extension = os.path.join(AllVideos_Path, '*_C.txt')
        for f in glob.glob(file_dir_extension):
            if not f.startswith('.'):
                yield os.path.basename(f)

    All_Videos=sorted(listdir_nohidden(AllVideos_Path))
    All_Videos.sort()
    AllFeatures = []  # To store C3D features of a batch
    print("Loading Abnormal videos Features...")

    Video_count=-1
    VideoLabels = []
    temp = ""
    ###################### READ RANDOMLY ABNORMAL FEATURE 32x512 #################
    for iv in Abnor_list_iter:
        Video_count=Video_count+1
        VideoPath = os.path.join(AllVideos_Path, All_Videos[iv])
        temp = VideoPath.split('/')[1].split('_')[0]
        events = [get_name(temp) for _ in range(32)]
        VideoLabels.extend(events)
        f = open(VideoPath, "r")
        words = f.read().split()
        num_feat = int(len(words) / 512)

        # Number of features per video to be loaded.
        # In our case num_feat=32, as we divide the video into 32 segments. 

        count = -1;
        VideoFeatues = []
        
        for feat in range(0, num_feat):
            feat_row1 = np.float32(words[feat * 512:feat * 512 + 512])
            count = count + 1
            if count == 0:
                VideoFeatues = feat_row1
            if count > 0:
                VideoFeatues = np.vstack((VideoFeatues, feat_row1))

        if Video_count == 0:
            AllFeatures = VideoFeatues
        if Video_count > 0:
            AllFeatures = np.vstack((AllFeatures, VideoFeatues))

    return AllFeatures,VideoLabels


In [0]:
AbnormalPath = os.path.join('Anomaly/')  # Path of abnormal already computed C3D features
image_list,  label_list =load_dataset_Train_batch(AbnormalPath)  # Load normal and abnormal video C3D features

Loading Abnormal videos Features...
Explosion018


In [0]:
image_size = len(image_list)
label_binarizer = LabelBinarizer()
image_labels = label_binarizer.fit_transform(label_list)
# pickle.dump(label_binarizer,open('label_transform.pkl', 'wb'))
n_classes = len(label_binarizer.classes_)

In [0]:
np_image_list = np.array(image_list, dtype=np.float16) / 225.0
np_image_list.shape

(4608, 512)

In [0]:
print("[INFO] Spliting data to train, test")
x_train, x_test, y_train, y_test = train_test_split(np_image_list, image_labels, test_size=0.35, random_state = 2) 

[INFO] Spliting data to train, test


In [0]:
from sklearn.tree import DecisionTreeClassifier
clf = DecisionTreeClassifier(random_state=0)
clf.fit(x_train, y_train)

print("Accuracy: ", clf.score(x_test,y_test))

Accuracy:  0.7947923124612524


In [0]:
from sklearn.neighbors import KNeighborsClassifier
neigh = KNeighborsClassifier(n_neighbors=3)
neigh.fit(x_train, y_train)
print("Accuracy: ", neigh.score(x_test,y_test))

Accuracy:  0.9869807811531308


Accuracy:  0.10477371357718537
